In [2]:
import csv
import pandas as pd
import numpy as np
import sklearn

from pymongo import MongoClient
global ALL_COLORS
ALL_COLORS = ['W', 'U', 'B', 'R', 'G']

PATH = './data/draft_data_public.MKM.PremierDraft.csv'

In [3]:
client = MongoClient()
cards_en = client.scryfall.cards_en

In [98]:
def get_card(cards_en, card_name, set=None):
    card = cards_en.find_one({'name': card_name})

    if card is None:
        # separate this query since it will make it slower to combine
        query = {            
            'card_faces': {
                '$elemMatch': {
                    'name': card_name
                }
            },
        }

        card = cards_en.find_one(query)

    if card is None:
        error_message = 'No match for card name {}'.format(card_name)
        print(error_message)

    return(card)

def pack_vector(row, pack_idx_min, names):
    return np.array([int(x) for x in row[pack_idx_min:pack_idx_min + len(names)]])

In [57]:
with open(PATH) as csvfile:
    draft_data = csv.reader(csvfile)
    columns = next(draft_data)
    names = [col.split('pack_card_')[1] for col in columns if 'pack_card_' in col]
    pick_idx = columns.index('pick')
    pack_idx_min = columns.index('pack_card_A Killer Among Us')
    pool_idx_min = columns.index('pool_A Killer Among Us')

In [148]:
alsa_filepath = './data/alsa.npy'
# calculate ALSA

# last_seen = np.zeros(len(names))
# counts = np.zeros(len(names))
# with open(PATH) as csvfile:
#     draft_data = csv.reader(csvfile)
#     next(draft_data)
#     draft_id = None
#     pack_num = None
#     row_last_seen = np.zeros(len(names))
    
#     for row in draft_data:
#         if draft_id != row[columns.index('draft_id')] or pack_num != row[columns.index('pack_number')]:
#             last_seen += row_last_seen
#             counts += np.where(row_last_seen, 1, 0)
#             draft_id = row[columns.index('draft_id')]
#             pack_num = int(row[columns.index('pack_number')])
#             row_last_seen = np.zeros(len(names))
#         pack = pack_vector(row, pack_idx_min, names)
#         np.putmask(row_last_seen, pack, int(row[columns.index('pick_number')]))
        
# alsa = last_seen / counts
# alsa_series = pd.Series(alsa, index=names)
# alsa_series = alsa_series[pd.notna(alsa)]

# with open(alsa_filepath, 'wb') as f:
#     np.save(f, alsa)

alsa = np.load(alsa_filepath)

In [89]:
def pip_vector(cards_en, card_name):
    card = get_card(cards_en, card_name)
    if card is None:
        print(card_name + ' Not Found')
    mana_cost = card['mana_cost']
    return np.array([mana_cost.count(f'{{{color}}}') for color in ALL_COLORS])  \
        + 1/2 * np.array([mana_cost.count(f'/{color}') for color in ALL_COLORS])\
        + 1/2 * np.array([mana_cost.count(f'{color}/') for color in ALL_COLORS])

def pool_pips(pip_matrix, pool):
    return np.matmul(pool.reshape(1,326), pip_matrix)

def pool_indicator(pool_pip_matrix, color_index, k):
    # does the pool have at least k pips of color ALL_COLORS[color_index]
    return float((pool_pip_matrix >= k)[0][color_index])
      
pip_matrix = np.array([pip_vector(cards_en, name) for name in names])